In [2]:
from octo.octo import OvercompleteBasis
from octo.basis import PixelBasis2D, CosineBasis2D
from octo.measurement import PathIntegral

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize